---

**APRP: 4. Parameter and Structure Learning**

---


**Contents:**

--------
1. Short context
2. Parameter Learning in a Bayes Net
3. Structure Learning in a Bayes Net




### 1. Short context

So far we have been focused on how Bayesian networks efficiently encode a probability distribution over a set of variables.
This Assignment will be about obtaining a Bayesian network, given a set of sample data. First we focus on the problem of **parameter Learning** for a given network and secondly on **learning the structure** itself.




### 2. Parameter Learning in a Bayes Net
## 2.1. Introductory example


In [1]:
!pip install pgmpy
#Based on https://pgmpy.org/detailed_notebooks/10.%20Learning%20Bayesian%20Networks%20from%20Data.html
import numpy as np
import pandas as pd
from pgmpy.models import BayesianModel
from pgmpy.estimators import BayesianEstimator
from pgmpy.estimators import MaximumLikelihoodEstimator

# Model
data = pd.DataFrame(data={'fruit': ["banana", "apple", "banana", "apple", "banana","apple", "banana",
                                    "apple", "apple", "apple", "banana", "banana", "apple", "banana",],
                          'tasty': ["yes", "no", "yes", "yes", "yes", "yes", "yes",
                                    "yes", "yes", "yes", "yes", "no", "no", "no"],
                          'size': ["large", "large", "large", "small", "large", "large", "large",
                                    "small", "large", "large", "large", "large", "small", "small"]})
model = BayesianModel([('fruit', 'tasty'), ('size', 'tasty')]) 


# Using a MLE estimator to obtain the CPDs tables
model.fit(data, estimator=MaximumLikelihoodEstimator)
for cpd in model.get_cpds():
    print(cpd)

# Using a Bayesian estimator to obtain the CPDs tables
model.fit(data, estimator=BayesianEstimator, prior_type="BDeu")
for cpd in model.get_cpds():
    print(cpd)



[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


/Users/marianapaulino/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,
/Users/marianapaulino/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/base/tsa_model.py:7: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import (to_datetime, Int64Index, DatetimeIndex, Period,


+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+--------------+-----+---------------+
| fruit      | fruit(apple) | ... | fruit(banana) |
+------------+--------------+-----+---------------+
| size       | size(large)  | ... | size(small)   |
+------------+--------------+-----+---------------+
| tasty(no)  | 0.25         | ... | 1.0           |
+------------+--------------+-----+---------------+
| tasty(yes) | 0.75         | ... | 0.0           |
+------------+--------------+-----+---------------+
+-------------+----------+
| size(large) | 0.714286 |
+-------------+----------+
| size(small) | 0.285714 |
+-------------+----------+
+---------------+-----+
| fruit(apple)  | 0.5 |
+---------------+-----+
| fruit(banana) | 0.5 |
+---------------+-----+
+------------+---------------------+-----+--------------------+
| fruit      | fruit(apple)        | ... | fruit(banana)      |
+------------+---------------

/Users/marianapaulino/opt/anaconda3/lib/python3.8/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


## 2.2. Questions
Based on the previous code provide an answer the following questions:

1. **For MLE, why small bananas are not tasty?**

If we analyse the data, there are two small bananas, and one of them is classified as tasty. Maximum likelihood estimator may have estimated that small bananas are not tasty, as the number of small bananas that are classified as not tasty is higher than the number of small bananas that are classified as tasty.

2. **Why the CPDs tables for MLE and Bayesian estimators differ?**

On one hand Bayesian estimation involve the incorporation of prior knowledge into the probability estimation process, this estimation tends to produce smoother and more realistic probability distributions (CPDs) especially for small sample sizes. On the other hand, MLE assumes an uniform prior distribution and may overfit the data and produce unrealistic probability distributions (CPDs) especially when there is little data available. The differences in the CPDs origined by these two estimations are origined by the prior information they recquire.

3. **What is the goal of the prior_type?**

prior_type="BDeu" is an argument used to obtain the CPDs (Conditional Probability Distributions) tables in a Bayesian Network. It specifies the type of prior probability used in the Bayesian Estimator.
BDeu (Bayesian Dirichlet equivalent uniform) prior type is a commonly used type prior for Bayesian networks. It is a smoothing technique that avoids probabilities to be zero or one. It allows the Bayesian Estimator to consider the possibility of unseen evidence, avoiding overfitting data.

### 3. Structure Learning
## 3.1. Scoring function. Example 1:

In [2]:
from pgmpy.estimators import BDeuScore, K2Score, BicScore

bdeu = BDeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

model1 = BayesianModel([('fruit', 'tasty'), ('size', 'tasty')]) # fruit -> tasty <- size
model2 = BayesianModel([('tasty', 'fruit'), ('tasty', 'size')]) # fruit <- tasty -> size

print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))


-30.12792467904587
-30.3772093643128
-32.859257093436106
-29.99714276768256
-30.551081866620226
-32.45409104969264


/Users/marianapaulino/opt/anaconda3/lib/python3.8/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


## 3.2. Scoring function. Example 2

In [3]:
# create random data sample with 3 variables, where Z is dependent on X, Y:
data = pd.DataFrame(np.random.randint(0, 4, size=(5000, 2)), columns=list('XY'))
data['Z'] = data['X'] + data['Y']

bdeu = BDeuScore(data, equivalent_sample_size=5)
k2 = K2Score(data)
bic = BicScore(data)

model1 = BayesianModel([('X', 'Z'), ('Y', 'Z')])  # X -> Z <- Y
model2 = BayesianModel([('X', 'Z'), ('X', 'Y')])  # Y <- X -> Z


print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))

print(bdeu.score(model2))
print(k2.score(model2))
print(bic.score(model2))

/Users/marianapaulino/opt/anaconda3/lib/python3.8/site-packages/pgmpy/models/BayesianModel.py:8: FutureWarning: BayesianModel has been renamed to BayesianNetwork. Please use BayesianNetwork class, BayesianModel will be removed in future.
  warnings.warn(


-13938.443992489887
-14329.246642608294
-14294.48331237518
-20901.906399844847
-20928.74658227
-20945.950639033526


## 3.3. Putting all toghether.

Example with HillClimbSearch and BicScore.

In [4]:
from pgmpy.estimators import HillClimbSearch

# create some data with dependencies
data = pd.DataFrame(np.random.randint(0, 5, size=(2500, 8)), columns=list('ABCDEFGH'))
data['A'] += data['B'] + data['C']
data['H'] = data['G'] - data['A']

hc = HillClimbSearch(data)
best_model = hc.estimate(scoring_method=BicScore(data))
print(best_model.edges())


  0%|          | 5/1000000 [00:00<25:27:50, 10.91it/s]

[('A', 'B'), ('A', 'C'), ('G', 'A'), ('H', 'A'), ('H', 'G')]


## 3.4. Questions
Based on the previous code provide an answer the following questions:

1. **Analyse the results from 3.1 and 3.2**

**3.1** 
In this dataset we have three columns 'fruit', 'tasty', and 'size' in this dataset we are in understanding the bayesian network structure using three different scoring functions (BDeuScore, K2Score, and BicScore) to score two different network structures. The first structure represents the dependencies as 'fruit' -> 'tasty' <- 'size', and the second represents the dependencies 'tasty' <- 'fruit' -> 'size'.

In the first model the scores were: -30.12792467904587 (BDeuScore), -30.3772093643128 (K2Score), and -32.859257093436106 (BicScore). When we applied the same scoring functions to the second, we obtained the following scores: -29.99714276768256 (BDeuScore), -30.551081866620226 (K2Score), and -32.45409104969264 (BicScore).
The results suggest that the second model is a better representation of the dependencies this happens because of the directions of the dependencies in this model because the direction of the dependencies in the second model reflects the causal relationships better between the variables.

**3.2** 
In the second dataset we have three variables 'X', 'Y', and 'Z'. Where 'Z' is dependent on 'X' and 'Y'. The goal is to understand the Bayesian network structure that best represents the dependencies between these variables. We used the same three scoring functions as before to score two different network structures. The first structure (model 1) represents the dependencies as 'X' -> 'Z' <- 'Y', and the second structure (model 2) represents the dependencies as 'Y' <- 'X' -> 'Z'. 

When applying the scoring functions to the first model, we obtained the following scores: -13940.064774913622 (BDeuScore), -14330.964883079389 (K2Score), and -14296.100426036432 (BicScore). And the second model obtained the following scores: -20911.9573808926 (BDeuScore), -20938.776897222077 (K2Score), and -20955.99232141435 (BicScore).
The first is a better representation of the dependencies in the data because the direction of the dependencies in model1 better reflects the causal relationships between the variables.

2. **Analyse the structure of the obtained network.**

**3.1**
The obtained structure has two nodes representing the variables "fruit" and "size", both have directed edge to a third node representing the variable "tasty". This structure indicates that both "fruit" and "size" have a direct influence on the "tasty" variable, and that there is no direct relationship between "fruit" and "size".

**3.2**
The obtained network structure has three nodes representing the variables "X", "Y", and "Z". There are two possible structures, the obtained scores suggest that the model where "X" is the parent of both "Y" and "Z" is a better fit for the data than the model where "X" is only the parent of "Z". This structure indicates that both "X" and "Y" have a direct influence on "Z", but that there is no direct relationship between "X" and "Y". Additionally, "X" is the parent of "Y", indicating that "X" has a direct influence on "Y".


### 4. Chalenge


1. Consider the file cancer.bif with the cancer disease BN already used.
2. Load the network and use the method *inference.likelihood_weighted_sample* to sample a minimumm of 5000 examples.
3. Calculate the CPDs and compare with the original ones.
4. Apply diferent strategies (scores and parameters) to determine a BN structure. Compare with the original.








In [5]:
from pgmpy.readwrite import BIFWriter
from pgmpy.readwrite import BIFReader


asian_model = BIFReader("asian_model4.bif").get_model()

In [6]:
# CPDs Originais

cpds_originais = asian_model.get_cpds()
for cpd in asian_model.get_cpds():
    print(cpd)


+---------+------+
| asia(0) | 0.99 |
+---------+------+
| asia(1) | 0.01 |
+---------+------+
+----------+----------+----------+
| smoke    | smoke(0) | smoke(1) |
+----------+----------+----------+
| bronc(0) | 0.7      | 0.4      |
+----------+----------+----------+
| bronc(1) | 0.3      | 0.6      |
+----------+----------+----------+
+---------+-----------+-----------+-----------+-----------+
| either  | either(0) | either(0) | either(1) | either(1) |
+---------+-----------+-----------+-----------+-----------+
| bronc   | bronc(0)  | bronc(1)  | bronc(0)  | bronc(1)  |
+---------+-----------+-----------+-----------+-----------+
| dysp(0) | 0.9       | 0.3       | 0.2       | 0.1       |
+---------+-----------+-----------+-----------+-----------+
| dysp(1) | 0.1       | 0.7       | 0.8       | 0.9       |
+---------+-----------+-----------+-----------+-----------+
+-----------+---------+---------+---------+---------+
| tub       | tub(0)  | tub(0)  | tub(1)  | tub(1)  |
+-----------

In [7]:
from pgmpy.sampling import BayesianModelSampling as BSM

samples = BSM(asian_model).likelihood_weighted_sample(size=5000)

Generating for node: dysp: 100%|██████████| 8/8 [00:00<00:00, 81.62it/s]


In [8]:
# Using a MLE estimator to obtain the CPDs tables
asian_model.fit(samples, estimator=MaximumLikelihoodEstimator)
for cpd in asian_model.get_cpds():
    print(cpd)


+---------+--------+
| asia(0) | 0.9892 |
+---------+--------+
| asia(1) | 0.0108 |
+---------+--------+
+----------+--------------------+--------------------+
| smoke    | smoke(0)           | smoke(1)           |
+----------+--------------------+--------------------+
| bronc(0) | 0.6956865848832607 | 0.4055802668823292 |
+----------+--------------------+--------------------+
| bronc(1) | 0.3043134151167392 | 0.5944197331176708 |
+----------+--------------------+--------------------+
+---------+---------------------+-----+---------------------+
| bronc   | bronc(0)            | ... | bronc(1)            |
+---------+---------------------+-----+---------------------+
| either  | either(0)           | ... | either(1)           |
+---------+---------------------+-----+---------------------+
| dysp(0) | 0.9004559270516718  | ... | 0.12727272727272726 |
+---------+---------------------+-----+---------------------+
| dysp(1) | 0.09954407294832827 | ... | 0.8727272727272727  |
+---------+---

In [9]:
# Using a Bayesian estimator to obtain the CPDs tables
asian_model.fit(samples, estimator=BayesianEstimator, prior_type="BDeu")
for cpd in asian_model.get_cpds():
    print(cpd)

+---------+-----------+
| asia(0) | 0.988711  |
+---------+-----------+
| asia(1) | 0.0112887 |
+---------+-----------+
+----------+--------------------+--------------------+
| smoke    | smoke(0)           | smoke(1)           |
+----------+--------------------+--------------------+
| bronc(0) | 0.6954931804704487 | 0.4056756210866492 |
+----------+--------------------+--------------------+
| bronc(1) | 0.3045068195295513 | 0.5943243789133509 |
+----------+--------------------+--------------------+
+---------+---------------------+-----+---------------------+
| bronc   | bronc(0)            | ... | bronc(1)            |
+---------+---------------------+-----+---------------------+
| either  | either(0)           | ... | either(1)           |
+---------+---------------------+-----+---------------------+
| dysp(0) | 0.9002658311971898  | ... | 0.13007518796992482 |
+---------+---------------------+-----+---------------------+
| dysp(1) | 0.09973416880281022 | ... | 0.8699248120300752  |

The original values in the CPDs tables are a bit higher (when the value is false) than the values in the Bayesian tables or MLE tables. In these tables the values are more similar and more close. The results of these two approaches don't vary much in the whole set of variables whether it is the xray, asia, smoke and the other ones. All the variables are categorical having binary values, assuming 0 when the value is false and 1 when the value is true meaning that the original table has higher predicting values when they are false.

In [10]:
from pgmpy.estimators import BDeuScore, K2Score, BicScore

bdeu = BDeuScore(samples, equivalent_sample_size=5)
k2 = K2Score(samples)
bic = BicScore(samples)

model1 = asian_model


print(bdeu.score(model1))
print(k2.score(model1))
print(bic.score(model1))



-11452.310645964219
-11451.207067057912
-11452.432455260458


In [11]:
from pgmpy.estimators import HillClimbSearch


hc = HillClimbSearch(samples)
best_model = hc.estimate(scoring_method=BicScore(samples))
print(best_model.edges())

  0%|          | 9/1000000 [00:01<39:27:01,  7.04it/s] 

[('bronc', 'smoke'), ('dysp', 'bronc'), ('either', 'lung'), ('either', 'xray'), ('either', 'tub'), ('either', 'dysp'), ('either', 'bronc'), ('lung', 'tub'), ('lung', 'smoke')]
